<a href="https://colab.research.google.com/github/mertsahinnn/Yapay-Zeka-Tabanli-Ses-Sistemlerinde-Gurlultu-Engelleme/blob/master/Yapay_Zeka_Tabanli_Ses_Sistemlerinde_Gurultu_Engelleme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
import shutil
import os

In [ ]:
# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive'),

Mounted at /content/drive


(None,)

In [ ]:
"""
Kaggle'dan veri setilerini drive kaydetmek için aşağıdaki kodları çalıştırın.

"""

# Kaggle'dan veri setini indir
path = kagglehub.dataset_download("truthisneverlinear/turkish-speech-corpus")

# Hedef klasör adını belirle
dataset_name = "turkish-speech-corpus/train"
destination_path_in_drive = os.path.join("/content/drive/MyDrive/turkish_corpus/", dataset_name)

# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive'),

# Veri setini Drive'a kopyala
# shutil.copytree(source, destination)
# Eğer hedef klasör zaten varsa hata verir, bu yüzden önceden kontrol edip silebiliriz.
if os.path.exists(destination_path_in_drive):
    shutil.rmtree(destination_path_in_drive) # Dikkat: Hedef klasörü ve içeriğini siler!

shutil.copytree("/root/.cache/kagglehub/datasets/truthisneverlinear/turkish-speech-corpus/versions/1/ISSAI_TSC_218/Train", destination_path_in_drive)

print(f"Veri seti Drive'a kaydedildi: {destination_path_in_drive}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Veri seti Drive'a kaydedildi: /content/drive/MyDrive/turkish_corpus/turkish-speech-corpus/train


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random

def summarize_audio_dataset(folder_path, num_samples_to_plot=3):
    """
    Belirtilen klasördeki tüm ses dosyalarını tarar, genel bir özet çıkarır
    ve rastgele seçilen bazı dosyaların dalga formu ile spektrogramını çizer.

    Args:
        folder_path (str): Ses dosyalarının bulunduğu klasörün yolu.
        num_samples_to_plot (int): Dalga formu ve spektrogramı çizilecek rastgele dosya sayısı.
    """
    if not os.path.isdir(folder_path):
        print(f"Hata: Klasör bulunamadı - {folder_path}")
        return

    print(f"--- '{folder_path}' Klasöründeki Ses Dosyalarını Analiz Ediliyor ---")

    audio_data = []
    supported_extensions = ['.wav', '.mp3', '.flac', '.ogg'] # Desteklenen ses formatları

    file_paths = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in supported_extensions):
                file_paths.append(os.path.join(root, file))

    if not file_paths:
        print("Belirtilen klasörde veya alt klasörlerinde desteklenen ses dosyası bulunamadı.")
        return

    total_files = len(file_paths)
    print(f"Toplam {total_files} ses dosyası bulundu.")

    # Her bir dosya için bilgi topla
    for i, file_path in enumerate(file_paths):
        print(f"İşleniyor: {os.path.basename(file_path)} ({i+1}/{total_files})")
        try:
            y, sr = librosa.load(file_path, sr=None)

            duration = librosa.get_duration(y=y, sr=sr)
            max_amplitude = np.max(np.abs(y))
            num_channels = y.shape[0] if len(y.shape) > 1 else 1 # Stereo mu mono mu kontrolü

            audio_data.append({
                'Dosya Adı': os.path.basename(file_path),
                'Yol': file_path,
                'Süre (sn)': duration,
                'Örnekleme Hızı (Hz)': sr,
                'Kanal Sayısı': num_channels,
                'Maksimum Genlik': max_amplitude
            })
        except Exception as e:
            print(f"Hata: '{os.path.basename(file_path)}' dosyası işlenirken bir sorun oluştu: {e}")
            audio_data.append({
                'Dosya Adı': os.path.basename(file_path),
                'Yol': file_path,
                'Süre (sn)': np.nan,
                'Örnekleme Hızı (Hz)': np.nan,
                'Kanal Sayısı': np.nan,
                'Maksimum Genlik': np.nan,
                'Hata': str(e)
            })

    # Verileri DataFrame'e dönüştür
    df = pd.DataFrame(audio_data)

    print("\n--- Genel Veri Seti Özeti ---")
    print(df.describe()) # Sayısal sütunların istatistiksel özetini göster

    print("\n--- Örnekleme Hızı Dağılımı ---")
    print(df['Örnekleme Hızı (Hz)'].value_counts())

    print("\n--- Kanal Sayısı Dağılımı ---")
    print(df['Kanal Sayısı'].value_counts())

    print(f"\nToplam ses süresi: {df['Süre (sn)'].sum():.2f} saniye (~{df['Süre (sn)'].sum()/3600:.2f} saat)")

    # Eksik/hatalı dosya kontrolü
    if df['Süre (sn)'].isnull().any():
        print("\n--- Hata veya Eksik Veri İçeren Dosyalar ---")
        print(df[df['Süre (sn)'].isnull()])


    # Rastgele seçilen dosyalardan görselleştirme
    print(f"\n--- Rastgele Seçilen {min(num_samples_to_plot, total_files)} Örnek Dosya Görselleştirmesi ---")

    # Hata içermeyen dosyalar arasından seçim yap
    successful_files = df.dropna(subset=['Süre (sn)'])
    if not successful_files.empty:
        samples_to_plot = successful_files['Yol'].sample(min(num_samples_to_plot, len(successful_files))).tolist()

        for file_path in samples_to_plot:
            try:
                y, sr = librosa.load(file_path, sr=None) # Tamamı yüklendi
                plt.figure(figsize=(12, 6))

                plt.subplot(2, 1, 1)
                librosa.display.waveshow(y, sr=sr)
                plt.title(f'{os.path.basename(file_path)} Dalga Formu')
                plt.xlabel('Zaman (saniye)')
                plt.ylabel('Genlik')
                plt.grid(True)

                D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
                plt.subplot(2, 1, 2)
                librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
                plt.colorbar(format='%+2.0f dB')
                plt.title(f'{os.path.basename(file_path)} Spektrogramı')
                plt.xlabel('Zaman (saniye)')
                plt.ylabel('Frekans (Hz)')
                plt.tight_layout()
                plt.show()

            except Exception as e:
                print(f"Görselleştirme sırasında hata: '{os.path.basename(file_path)}' - {e}")
    else:
        print("Görselleştirilecek uygun dosya bulunamadı (tüm dosyalar hatalı olabilir).")


# --- Kullanım Örneği ---
if __name__ == "__main__":


    # !!! BURAYI KENDİ VERİ KÜMENİZİN OLDUĞU KLASÖR YOLU İLE DEĞİŞTİRİN !!!
    your_audio_dataset_folder = "/content/drive/MyDrive/turkish_corpus/turkish-speech-corpus/"


    summarize_audio_dataset(your_audio_dataset_folder, num_samples_to_plot=3)

--- '/content/drive/MyDrive/turkish_corpus/turkish-speech-corpus/' Klasöründeki Ses Dosyalarını Analiz Ediliyor ---
